In [37]:
from blinker import signal

def send_sig(signal_name, **kwargs):
    s = signal("events")
    return s.send(signal_name,**kwargs)


events = signal("events")
# @events.connect_via("test")
# def on_test(sender, message: str):
#     print(f"Received notification: {message}")

@events.connect
def log(sender, **kwargs):
    yield f"Logging signal: Sender: {sender} Args: {kwargs}"
    print(f"Logging signal: Sender: {sender} Args: {kwargs}")

In [38]:
results = send_sig("test2", message="Hello")
# send_sig("test3", **{"a": 2, "b": 2})
# signal("test2").send("test2", message="Hello THERE!!")
results

Logging signal: Sender: test2 Args: {'message': 'Hello'}


[(<function __main__.log(sender, **kwargs)>, None),
 (<function __main__.log(sender, **kwargs)>,
  <generator object log at 0x7f710b2917e0>)]

In [3]:
class Notifcation:
    def __init__(self, message: str):
        events.send("test", message=message)


tester = Notifcation("Hello from Tester")

Logging signal: Sender: test Args: {'message': 'Hello from Tester'}
Received notification: Hello from Tester


In [4]:
Notifcation("Hello from Second Tester")


Logging signal: Sender: test Args: {'message': 'Hello from Second Tester'}
Received notification: Hello from Second Tester


In [5]:
from blinker import ANY, signal
from typing import TypeVar, Any, Callable
import collections.abc as c
test2 = signal("test2")

F = TypeVar("F", bound=c.Callable[..., Any])

def event(signal_name: str, sender: Any = ANY, weak: bool = True) -> c.Callable[[F], F]:
    sig = signal(signal_name)
    def decorator(fn):
        sig.connect(fn, sender, weak)
        return fn
    return decorator


In [ ]:
@event("test2")
def on_test(sender, message: str):
    print(f"Received notification2: {message}")

In [7]:
from blinker.base import make_id
test2.send("test2", message="Hello THERE!!")
for test in test2.receivers:
    print(test)
make_id(on_test)
test2.receivers[make_id(on_test)]

Received notification2: Hello THERE!!
140123505369440


<weakref at 0x7f710bbe58f0; to 'function' at 0x7f710bc32160 (on_test)>

In [8]:
from collections import deque

queue = deque([])  # list of tuples is fine

queue.extend([("job3", 14), ("job4", 15)])

print(queue)

while queue:
    item = queue.popleft()  # fast O(1)
    print("Processing:", item)
    
print(queue)

deque([('job3', 14), ('job4', 15)])
Processing: ('job3', 14)
Processing: ('job4', 15)
deque([])


In [9]:
signal("events").send("user", message="Hello")

Logging signal: Sender: user Args: {'message': 'Hello'}


[(<function __main__.log(sender, **kwargs)>, None)]

In [10]:
from uuid import uuid4
str(uuid4().hex)

'cd67d0db60074f7ab1df0852ab2c4aef'

In [11]:
class AttrDict(dict):
    "`dict` subclass that also provides access to keys as attrs"
    def __getattr__(self,k): return self[k] if k in self else None
    def __setattr__(self, k, v): (self.__setitem__,super().__setattr__)[k[0]=='_'](k,v)
    def __dir__(self): return super().__dir__() + list(self.keys())
    def copy(self): return AttrDict(**self)


signals = AttrDict(message="Hello")
signals.message
signals.name = "John"
signals.name or "No name"



'John'

In [24]:
sigi = signal("sigi")

@sigi.connect
def log(sender, **kwargs):
    print(f"Logging signal: Sender: {sender} Args: {kwargs}")

sigi.send()



TypeError: sync_gen() takes 0 positional arguments but 1 was given